In [1]:
cd ..

/home/jevans96/umd_icecube_analysis_tutorial


In [2]:
__author__ = 'John Evans'
__copyright__ = ''
__credits__ = ['John Evans', 'Jason Fan', 'Michael Larson']
__license__ = 'Apache License 2.0'
__version__ = '0.0.1'
__maintainer__ = 'John Evans'
__email__ = 'jevans96@umd.edu'
__status__ = 'Development'

"""
Docstring
"""
from typing import Optional

import numpy as np
import healpy as hp
from i3pubtools import tools
from i3pubtools import sources
from i3pubtools.llhs import ps_llh
# from test import test_data
import glob
import time
import itertools
from scipy import stats
from matplotlib import pyplot as plt
import cProfile

np.random.seed(0)

In [3]:
# Load in both the simulation and the data

# # Where is the dataset stored?
dataset = '/data/mjlarson/data/'

# Read in all of the data files
data_files = dataset + '/IC86_*exp.npy'
data = tools.read(glob.glob(data_files))

# Read in all of the MC files 
sim_files = dataset + '/IC86*MC*npy'
sim = tools.read(glob.glob(sim_files))

# trim data
data = data[:1000]

# data_size = 10000
# sim_size = 100000

# data = test_data.get_random_data(data_size)
# sim = test_data.get_random_sim(sim_size)

# Set the angular error floor to 0.2 degrees
data['angErr'][data['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)
sim['angErr'][sim['angErr']<np.deg2rad(0.2)] = np.deg2rad(0.2)


# Show the possible keys available here:
print('Events available in simulation: {}'.format(len(sim)))
print('Keys available in simulation:')
print(sorted(sim.dtype.names))
print()

print('Events available in data: {}'.format(len(data)))
print('Keys available in data:')
print(sorted(data.dtype.names))

Events available in simulation: 11595414
Keys available in simulation:
['angErr', 'azi', 'dec', 'event', 'logE', 'ow', 'ra', 'run', 'subevent', 'time', 'trueDec', 'trueE', 'trueRa', 'zen']

Events available in data: 1000
Keys available in data:
['angErr', 'azi', 'dec', 'event', 'logE', 'ra', 'run', 'subevent', 'time', 'zen']


In [4]:
# # Also load in the 'GoodRunList' (GRL), a file that tells
# # us when the detector was taking good data. 
grl_files = dataset + '/GRL/IC86_*exp.npy'
grl = tools.read(glob.glob(grl_files))
# grl = test_data.get_random_grl(data)


# Show the keys available in the GRL
print('Keys available in the GoodRunList:')
print(sorted(grl.dtype.names))
print(np.min(grl['start']))
print(np.max(grl['stop']))
gammas = np.arange(-4.25, -0.5, 0.25)
bins   = np.array([np.linspace(-1, 1, 51), # sin(dec)
                   np.linspace(1, 8, 51)]) # logE

Keys available in the GoodRunList:
['events', 'livetime', 'run', 'start', 'stop']
55694.99047453704
58309.08234953694


In [5]:
# Try this with a gaussian signal + uniform background.
outfile = './test/test_outputs/ps_llh-test.npy'
infile = './test/test_outputs/ps_llh-test.npy'

args = (data, sim, grl, gammas, bins)

kwargs = {'source': {'ra':np.pi/2, 'dec':np.pi/6}}

In [ ]:
# Run this block once

#cProfile.run('ps_llh.PsFlareLLH(*args, outfile = outfile, **kwargs)')
llh_object = ps_llh.PsLLH(*args, outfile = outfile, **kwargs)

In [6]:
# Afterwards, run this one to save time
llh_object = ps_llh.PsLLH(*args, infile = infile, **kwargs)

In [7]:
# Produce a single trial. Here, we pass a 'seed'
# value to the function so that we can reproduce
# the trial later if we want to. If you want it to
# be random, set seed=None.
seed = 0
flux_norm = 1e-10
#cProfile.run('llh_object.produce_trial(flux_norm=flux_norm, random_seed=seed)')
#%timeit llh_object.produce_trial(flux_norm=flux_norm, random_seed=seed)
trial = llh_object.produce_trial(flux_norm=flux_norm, random_seed=seed, verbose=True)
print('Got {} events for this trial'.format(len(trial)))

print('Times:')
print(trial)

number of background events: 7429
number of signal events: 25
Got 7429 events for this trial
Times:
[(120028,  839634, 0, 4.01747511, -0.99286837, 3.35256417, 0.57782915, 56043.42668248, 4.94667356, 0.00349066)
 (120028,  839634, 0, 1.82202897, -0.99286837, 3.35256417, 0.57782915, 56043.42668248, 4.94667356, 0.00349066)
 (120028,  839634, 0, 0.15700388, -0.99286837, 3.35256417, 0.57782915, 56043.42668248, 4.94667356, 0.00349066)
 ...
 (120165, 1676923, 0, 2.16234462,  0.42893458, 1.90703768, 2.00076047, 56064.45676784, 3.51292094, 0.02860901)
 (120165, 1676923, 0, 2.58828939,  0.42893458, 1.90703768, 2.00076047, 56064.45676784, 3.51292094, 0.02860901)
 (120165, 1676923, 0, 1.22216895,  0.42893458, 1.90703768, 2.00076047, 56064.45676784, 3.51292094, 0.02860901)]


In [18]:
def plot_skymap(
    events: np.ndarray,
    outdir: Optional[str],
    title: str = '',
    nside: int = 32,
    hist_type: str = 'counts',
    verbose: bool = False,
) -> None:
    """Function info...
    
    More function info...
    
    Args:
        events:
        outdir:
        title:
        nside:
        verbose:
    """
    npix = hp.nside2npix(nside)
    theta = np.pi/2 - events['dec']
    phi = events['ra']
    indices = hp.ang2pix(nside, theta, phi)
    hpx_map = np.zeros(npix, dtype=int)
    if hist_type == 'counts':
        idx, counts = np.unique(indices, return_counts=True)
        hpx_map[idx] = counts
        title = '_'.join([title, 'counts'])
    else: # if hist_type == 'ts'
        title = '_'.join([title, 'ts'])
        for idx in np.unique(indices):
            hpx_map[idx] = np.sum(np.where(indices == idx, events['ts'], 0))
            
    hp.mollview(hpx_map, title=title)
    
    if outdir is not None:
        outfile_prefix = ''.join([outdir, title])
        plt.savefig('_'.join([outfile_prefix, f'{nside}_plot.pdf']))
    
    if verbose:
        plt.show()
    
    plt.close()

In [9]:
source_dict = sources.sources
flux_norms = np.logspace(-10, -8, 3)
seed = 0
outdir = './test/test_outputs/'

print(source_dict)

for (name, source), flux_norm in itertools.product(source_dict.items(), flux_norms):
    llh_object.source = source
    print(f'Generating trial: Source = {name}, N = {flux_norm}')
    trial = llh_object.produce_trial(flux_norm=flux_norm, random_seed=seed, verbose=True)
    print('Calculating ts...', end='')
    best_fit = llh_object.evaluate_ts(trial)
    ts = llh_object.get_individual_ts(trial, best_fit['ns'], best_fit['gamma'])
    print('done')
    title = f'{name}_N-{flux_norm}_{time.perf_counter()}'
    np.save(''.join([outdir, title, '_trial.npy']), ts)

print('Generating background trial:')
trial = llh_object.produce_trial(flux_norm=0, random_seed=seed, verbose=True)
print('Calculating ts...', end='')
best_fit = llh_object.evaluate_ts(trial)
ts = llh_object.get_individual_ts(trial, best_fit['ns'], best_fit['gamma'])
print('done')
title = f'Background_{time.perf_counter()}'
np.save(''.join([outdir, title, '_trial.npy']), ts)

{'crab_nebula': {'ra': 1.4596726677614609, 'dec': 0.3842255081802917}, 'txs': {'ra': 1.3501550420509332, 'dec': 0.09936418146033345}, 'm77': {'ra': 0.709823297846581, 'dec': -0.00023193486504280206}, 'mgro1908': {'ra': 5.008658620910728, 'dec': 0.10940790341598898}, 'sag_a_star': {'ra': 4.649850989853494, 'dec': -0.50628180298921}, 'mag_l': {'ra': 1.411862281785163, 'dec': -1.217473812275889}, 'mag_s': {'ra': 0.23015075069631893, 'dec': -1.2710990535434146}}
Generating trial: Source = crab_nebula, N = 1e-10
number of background events: 7429
number of signal events: 25
Calculating ts...done
Generating trial: Source = crab_nebula, N = 1e-09
number of background events: 7429
number of signal events: 285
Calculating ts...done
Generating trial: Source = crab_nebula, N = 1e-08
number of background events: 7429
number of signal events: 2966
Calculating ts...done
Generating trial: Source = txs, N = 1e-10
number of background events: 7429
number of signal events: 25
Calculating ts...done
Genera

In [14]:
trial_files = ''.join([outdir, '*_trial.npy'])
nside = 64
for f in glob.glob(trial_files):
    events = np.load(f)
    title = f.split('/')[-1]
    plot_skymap(trial, outdir=outdir, title=title, nside=nside, hist_type='counts')

In [19]:
for f in glob.glob(trial_files):
    ts = np.load(f)
    title = f.split('/')[-1]
    plot_skymap(ts, outdir=outdir, title=title, nside=nside, hist_type='ts')